In [4]:
from itertools import combinations
import numpy as np
import numpy.linalg as la
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from statsmodels.stats.outliers_influence import variance_inflation_factor
from matplotlib_venn import venn3

import sys
sys.path.append('../mchem_functions/')
import multivariate_functions as mvf

In [5]:
nodes_by_edges = np.array([
    [1,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0],
    [1,0,1,0,0,0,0,0],
    [1,0,0,1,0,0,0,0],
    [0,1,0,0,1,0,0,0],
    [1,0,0,1,0,1,0,0],
    [0,1,0,0,1,0,1,0],
    [0,1,0,0,1,0,0,1]
])

In [6]:
eigvals, proportion_explained, U, F = mvf.PCA(nodes_by_edges)

In [8]:
pd.DataFrame(F)

,0,1,2,3,4,5,6
0,-0.637338,-0.348307,0.000000,-0.086654,-0.286597,-0.364266,0.125580
1,0.436618,-0.138237,0.000000,-0.711248,0.135973,0.012842,-0.177068
2,-0.753401,-0.889463,0.000000,0.275346,0.246468,0.136595,-0.040598
3,-0.960072,0.299727,0.000000,-0.022433,-0.411090,0.281991,-0.070950
4,0.906334,0.050902,0.000000,-0.088460,0.053174,0.154307,0.396394
5,-1.134905,0.765404,0.000000,0.071281,0.353529,-0.105743,0.022937
6,1.071382,0.129987,-0.707107,0.281084,-0.045728,-0.057863,-0.128148
7,1.071382,0.129987,0.707107,0.281084,-0.045728,-0.057863,-0.128148
